In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')

In [3]:
test = pd.read_csv('test.csv')

In [4]:
y_train = train.pop('target').values
train['source']='train'
test['source']='test'
df = pd.concat([train, test],ignore_index=True)
df = df.set_index('ID_code')
df.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,source
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914,train
train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518,train
train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965,train
train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996,train
train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104,train


In [5]:
categorical_columns = [x for x in df.dtypes.index if df.dtypes[x]=='object']
categorical_columns

['source']

In [6]:
for column in df.columns:
    if column !='source':
        df[column] = pd.qcut(df[column],3)
        df.head()      

In [7]:
df = pd.get_dummies(df)
df.head()

,"var_0_(0.188, 9.164]","var_0_(9.164, 11.968]","var_0_(11.968, 22.323]","var_1_(-15.043999999999999, -3.617]","var_1_(-3.617, 0.344]","var_1_(0.344, 10.377]","var_2_(2.116, 9.346]","var_2_(9.346, 11.92]","var_2_(11.92, 19.353]","var_3_(-0.0412, 5.79]",...,"var_197_(8.474, 9.345]","var_197_(9.345, 12.276]","var_198_(6.298, 14.562]","var_198_(14.562, 17.322]","var_198_(17.322, 26.538]","var_199_(-39.458999999999996, -8.267]","var_199_(-8.267, 2.364]","var_199_(2.364, 28.501]",source_test,source_train
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,1,0,0,1,0,0,0,1,0,1,...,1,0,1,0,0,0,1,0,0,1
train_1,0,1,0,1,0,0,0,0,1,1,...,1,0,0,0,1,0,1,0,0,1
train_2,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
train_3,0,1,0,0,1,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,1
train_4,0,1,0,0,1,0,0,0,1,0,...,0,1,0,0,1,1,0,0,0,1


In [8]:
X_train = df[df['source_train']==1].drop(['source_train','source_test'], axis=1).values
X_test = df[df['source_test']==1].drop(['source_train','source_test'], axis=1).values

In [10]:
import xgboost as xgb
model = xgb.XGBClassifier(max_depth=20, min_child_weight=1,  n_estimators=100,\
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.16, max_delta_step=0,
       max_depth=20, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1, verbose=1)

In [11]:
y_pred = model.predict(X_test)

In [12]:
sample_file = pd.read_csv('sample_submission.csv')
sample_file['target'] = y_pred
sample_file.to_csv('satander_submission4.csv', index=False)